In [5]:
import numpy as np # linear algebra
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/qwen3-finetuned/transformers/default/3/__huggingface_repos__.json
/kaggle/input/qwen3-finetuned/transformers/default/3/wandb/run-20250712_162007-as3dltlu/run-as3dltlu.wandb
/kaggle/input/qwen3-finetuned/transformers/default/3/wandb/run-20250712_162007-as3dltlu/logs/debug.log
/kaggle/input/qwen3-finetuned/transformers/default/3/wandb/run-20250712_162007-as3dltlu/logs/debug-internal.log
/kaggle/input/qwen3-finetuned/transformers/default/3/wandb/run-20250712_162007-as3dltlu/files/wandb-summary.json
/kaggle/input/qwen3-finetuned/transformers/default/3/wandb/run-20250712_162007-as3dltlu/files/config.yaml
/kaggle/input/qwen3-finetuned/transformers/default/3/wandb/run-20250712_162007-as3dltlu/files/output.log
/kaggle/input/qwen3-finetuned/transformers/default/3/wandb/run-20250712_162007-as3dltlu/files/requirements.txt
/kaggle/input/qwen3-finetuned/transformers/default/3/wandb/run-20250712_162007-as3dltlu/files/wandb-metadata.json
/kaggle/input/qwen3-finetuned/transformers/defaul

In [6]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == 

In [7]:
!pip install -q sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.4 MB/s eta 0:00:00


In [8]:
!pip install -q wandb

In [9]:
# imports
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset
import torch
import wandb
import torch.nn as nn
from torch.utils.data import DataLoader
from accelerate import Accelerator
from transformers.optimization import AdafactorSchedule
from transformers.optimization import get_cosine_schedule_with_warmup
from accelerate import Accelerator
from tqdm.auto import tqdm
import math
from transformers.optimization import Adafactor, get_scheduler
from typing import Dict, List, Tuple
import json
from datetime import datetime
from peft import LoraConfig , get_peft_model
from transformers import DataCollatorForSeq2Seq
import shutil
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler
import gc
torch.backends.cudnn.benchmark = True  # Optimize for P100
torch.backends.cudnn.deterministic = False  

2025-07-13 18:23:26.824964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752431007.005480      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752431007.059761      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
# connect wandb library for graphs
from kaggle_secrets import UserSecretsClient

secret = UserSecretsClient()
os.environ["WANDB_API_KEY"] = secret.get_secret("WANDB_KEY") 

wandb.login()   

wandb: Currently logged in as: 22cs3033 (22cs3033-rgipt-jais) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
# loading model and tokenizer into the RAM
model_name = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True )
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [12]:
# create the response on default pretrained model
prompt = "Dengue cases on the rise in city"
messages = [
    {"role": "system", "content": "You are a translator. who knows marathai and english language very well. Your task is convert given english sentence in marathi."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [13]:
response

'<think>\nOkay, the user wants me to translate the sentence "Dengue cases on the rise in city" into Marathi. Let me start by understanding the original sentence. The main components are "Dengue cases on the rise" and "in city". \n\nFirst, "Dengue cases" is a medical term, so I need to make sure it\'s translated accurately. In Marathi, "Dengue" is commonly known as "देंगू" or sometimes "देंगू अस्पर्त रेंस". Since it\'s a specific condition, maybe "देंगू" is better. \n\nNext, "cases on the rise" – "cases" here refers to the number of cases, so "संख्या" might be appropriate. The phrase "on the rise" could be "स्थायितीय रूप वार्ता". So combining these, "Dengue cases on the rise in city" becomes "देंगू अस्पर्त रेंस स्थायितीय रूप वार्ता वर्ता विश्व राज्य". \n\nWait, "in city" – "विश्व राज्य" is the general term for a country or region. But "city" could also be "महाराज्य" or "महाराज्य विश्व". Since the original is "on the rise in city", maybe "महाराज्य विश्व" is more specific. \n\nI should ch

In [14]:
# load the english to marathi translation data from huggingface
ds = load_dataset("anujsahani01/English-Marathi")

README.md:   0%|          | 0.00/206 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/621M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/243M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2637962 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/879321 [00:00<?, ? examples/s]

In [15]:
ds

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 2637962
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 879321
    })
})

In [16]:
ds['train'][10]

{'english': 'The funds collected are used for social purposes.',
 'marathi': 'त्या निधीचा वापर सामाजिक कार्यासाठी करतो.'}

In [17]:
# Dataset is too big so slice it accroding your computation resources
df_train = ds["train"].select(range(10000))
df_validation = ds["train"].select(range(10000 , 11000))
df_test = ds["test"].select(range(200))

In [18]:
# # Dataset is too big so slice it for quick experimentation
# df_train = ds["train"].select(range(500))
# df_validation = ds["train"].select(range(500 , 550))
# df_test = ds["test"].select(range(50))

In [19]:
# Qwen3-0.6B is Causal Language Models so we have finetuned it Instruction-output format. (instruction‑tuning)
def add_instruction(example):
    example["instruction"] = "Convert the English text into Marathi language."
    return example


df_train = df_train.map(add_instruction, batched=False)
df_validation = df_validation.map(add_instruction, batched=False)
df_test = df_test.map(add_instruction, batched=False)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [20]:
ds["train"].column_names

['english', 'marathi']

In [21]:
df_train[0]

{'english': 'Next few months are really crucial for us.',
 'marathi': 'पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.',
 'instruction': 'Convert the English text into Marathi language.'}

In [22]:
# Know about how tokenizer works
inputs = tokenizer('Next few months are really crucial for us.','पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.', 'Convert the English text into Marathi language.' )
inputs

{'input_ids': [5847, 2421, 3951, 525, 2167, 16587, 369, 601, 13, 86162, 72653, 149269, 43647, 91811, 14925, 97, 43647, 60096, 91217, 93948, 42311, 101, 34370, 14925, 228, 87244, 146113, 30484, 107, 31411, 116, 31411, 254, 43647, 14925, 244, 12619, 224, 86162, 91217, 93948, 79238, 30484, 97, 30484, 113, 86162, 12619, 224, 44179, 30484, 96, 14925, 228, 93948, 54784, 97, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [12012, 279, 6364, 1467, 1119, 2876, 66531, 4128, 13]}

In [23]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['Next',
 'Ġfew',
 'Ġmonths',
 'Ġare',
 'Ġreally',
 'Ġcrucial',
 'Ġfor',
 'Ġus',
 '.',
 'à¤ª',
 'à¥ģ',
 'à¤¢',
 'à¥Ģ',
 'à¤²',
 'Ġà¤',
 '¤',
 'à¥Ģ',
 'à¤¨',
 'Ġà¤®',
 'à¤¹',
 'à¤¿à¤',
 '¨',
 'à¥ĩ',
 'Ġà¤',
 'Ĩ',
 'à¤®',
 'à¤ļ',
 'à¥įà¤',
 '¯',
 'à¤¾à¤',
 '¸',
 'à¤¾à¤',
 'ł',
 'à¥Ģ',
 'Ġà¤',
 'ĸ',
 'à¥',
 'Ĥ',
 'à¤ª',
 'Ġà¤®',
 'à¤¹',
 'à¤¤',
 'à¥įà¤',
 '¤',
 'à¥įà¤',
 'µ',
 'à¤ª',
 'à¥',
 'Ĥ',
 'à¤°',
 'à¥įà¤',
 '£',
 'Ġà¤',
 'Ĩ',
 'à¤¹',
 'à¥ĩà¤',
 '¤',
 '.']

In [24]:
instruction = "Convert the English text into Marathi language."
src_text     = "Next few months are really crucial for us."
tgt_text     = "पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत."

# Prompt the model should *see*
prompt = f"{instruction} English: {src_text} Marathi:"
print(prompt)

inputs  = tokenizer(prompt, return_tensors="pt") 
print('Length of tokenize form of prompt:', inputs['input_ids'].size())


# Every token sequence in a causal‐LM fine‑tuning setup needs a clear “stop” signal so the model knows when it’s done generating. That’s exactly what tokenizer.eos_token_id is for

labels  = tokenizer(tgt_text, add_special_tokens=False).input_ids + [tokenizer.eos_token_id] #Here labes are tokenize form our marathi output

Convert the English text into Marathi language. English: Next few months are really crucial for us. Marathi:
Length of tokenize form of prompt: torch.Size([1, 23])


In [25]:
inputs

{'input_ids': tensor([[12012,   279,  6364,  1467,  1119,  2876, 66531,  4128,    13,  6364,
            25,  9295,  2421,  3951,   525,  2167, 16587,   369,   601,    13,
          2876, 66531,    25]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [26]:
# The length of tokenize input prompt( instrution + context) is 23

In [27]:
tokenizer.eos_token_id

151645

In [28]:
labels 

[86162,
 72653,
 149269,
 43647,
 91811,
 14925,
 97,
 43647,
 60096,
 91217,
 93948,
 42311,
 101,
 34370,
 14925,
 228,
 87244,
 146113,
 30484,
 107,
 31411,
 116,
 31411,
 254,
 43647,
 14925,
 244,
 12619,
 224,
 86162,
 91217,
 93948,
 79238,
 30484,
 97,
 30484,
 113,
 86162,
 12619,
 224,
 44179,
 30484,
 96,
 14925,
 228,
 93948,
 54784,
 97,
 13,
 151645]

In [29]:
len(labels) # 49 orignal + 1 tokenizer.eos_token_id = 50

50

In [30]:
# For a causal model, you feed it the entire sequence (prompt and target), because it learns by shifting that sequence one token at a time.
model_inputs = tokenizer(prompt + " " + tgt_text + tokenizer.eos_token,  return_tensors="pt")
print('Length of tokenize form of model_inputs:', model_inputs['input_ids'].size())  

# You need to know how many tokens correspond just to the instruction+source (the “prompt”) so that you can later mask them out in the labels.
prompt_len = len(tokenizer(prompt, add_special_tokens=False).input_ids)
print("Tokenized Prompt length:" , prompt_len)

#  Clone so you don’t overwrite the original inputs
labels = model_inputs["input_ids"].clone()


# For every position in the prompt, set label to -100  
# In Hugging Face Trainer, any label token = –100 is ignored by the loss function.
labels[:, :prompt_len] = -100        # works for a batch of size 1 or N
# print('Length that going to be masked' , len(labels))
model_inputs["labels"] = labels

Length of tokenize form of model_inputs: torch.Size([1, 73])
Tokenized Prompt length: 23


In [31]:
labels

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,  83636,  72653, 149269,  43647,
          91811,  14925,     97,  43647,  60096,  91217,  93948,  42311,    101,
          34370,  14925,    228,  87244, 146113,  30484,    107,  31411,    116,
          31411,    254,  43647,  14925,    244,  12619,    224,  86162,  91217,
          93948,  79238,  30484,     97,  30484,    113,  86162,  12619,    224,
          44179,  30484,     96,  14925,    228,  93948,  54784,     97,     13,
         151645]])

In [32]:
labels.size()

torch.Size([1, 73])

In [33]:
x = labels[0] 
num_ignored = (x == -100).sum().item()
num_not_ignored = (x != -100).sum().item()
print(num_ignored)
print(num_not_ignored)

23
50


In [34]:
model_inputs

{'input_ids': tensor([[ 12012,    279,   6364,   1467,   1119,   2876,  66531,   4128,     13,
           6364,     25,   9295,   2421,   3951,    525,   2167,  16587,    369,
            601,     13,   2876,  66531,     25,  83636,  72653, 149269,  43647,
          91811,  14925,     97,  43647,  60096,  91217,  93948,  42311,    101,
          34370,  14925,    228,  87244, 146113,  30484,    107,  31411,    116,
          31411,    254,  43647,  14925,    244,  12619,    224,  86162,  91217,
          93948,  79238,  30484,     97,  30484,    113,  86162,  12619,    224,
          44179,  30484,     96,  14925,    228,  93948,  54784,     97,     13,
         151645]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]]), 'labels': tensor([[  -100,   -100,   -100,

In [35]:
model_inputs['input_ids'].size()

torch.Size([1, 73])

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
class LoRAConfig:
    """Centralized configuration for LoRA fine-tuning"""
    
    # LoRA Parameters
    LORA_RANK = 8  
    LORA_ALPHA = 16  
    LORA_DROPOUT = 0.1  
    
    # Training Parameters
    LEARNING_RATE = 2e-4  
    BATCH_SIZE = 2
    GRADIENT_ACCUMULATION_STEPS = 3  
    NUM_EPOCHS = 1
    WARMUP_STEPS = 100
    MAX_GRAD_NORM = 1.0  
    
    # Optimization
    WEIGHT_DECAY = 0.01
    BETA1 = 0.9
    BETA2 = 0.999
    EPS = 1e-8
    
    # Monitoring and Saving
    EVAL_EVERY_N_STEPS = 1000
    SAVE_EVERY_N_STEPS = 2000
    MAX_CHECKPOINTS = 3
    
    # Data Processing
    MAX_LENGTH = 4096 
    PAD_TO_MULTIPLE_OF = 8 

config = LoRAConfig()

In [38]:
# this function convert string into vectors
def tokenize_function(examples):
    batch_input_ids = []
    batch_attention_mask = []
    batch_labels = []

    for instruction, english, marathi in zip(
            examples["instruction"], examples["english"], examples["marathi"]):

        # 1. Create the prompt (what model sees but doesn't train on)
        prompt_text = f"{instruction} English: {english} Marathi:"
        
        # 2. Create full text (prompt + target response)
        full_text = f"{prompt_text} {marathi}{tokenizer.eos_token}"
        
        # 3. Tokenize prompt separately to know where to mask
        prompt_tokens = tokenizer(
            prompt_text, 
            add_special_tokens=False, 
            truncation=True, 
            max_length=config.MAX_LENGTH//2,
        )
        
        # 4. Tokenize full text
        full_tokens = tokenizer(
            full_text,
            add_special_tokens=False,
            truncation=True,
            max_length=config.MAX_LENGTH,
            padding=False,
        )
        
        # 5. Create labels: -100 for prompt tokens (ignored in loss)
        prompt_length = len(prompt_tokens["input_ids"])
        labels = [-100] * prompt_length + full_tokens["input_ids"][prompt_length:]
        
        # 6. Ensure labels and input_ids have same length
        if len(labels) != len(full_tokens["input_ids"]):
            labels = labels[:len(full_tokens["input_ids"])]
        
        # 7. Keep as Python lists (no numpy, no tensors)
        batch_input_ids.append(full_tokens["input_ids"])
        batch_attention_mask.append(full_tokens["attention_mask"])
        batch_labels.append(labels)

    return {
        "input_ids": batch_input_ids,
        "attention_mask": batch_attention_mask,
        "labels": batch_labels,
    }

# Apply tokenization
print(" Tokenizing datasets")
train_tokenized = df_train.map(
    tokenize_function, 
    batched=True, 
    num_proc=4,
    remove_columns=df_train.column_names,
    desc="Tokenizing training data"
)

test_tokenized = df_test.map(
    tokenize_function, 
    batched=True, 
    num_proc=4,
    remove_columns=df_test.column_names,
    desc="Tokenizing test data"
)

validation_tokenized = df_validation.map(
    tokenize_function, 
    batched=True, 
    num_proc=4,
    remove_columns=df_validation.column_names,
    desc="Tokenizing validation data"
)

print(f"Tokenization complete!")
print(f"Train: {len(train_tokenized)} examples")
print(f"Validation: {len(validation_tokenized)} examples") 
print(f"Test: {len(test_tokenized)} examples")

 Tokenizing datasets


Tokenizing training data (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing test data (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing validation data (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenization complete!
Train: 10000 examples
Validation: 1000 examples
Test: 200 examples


In [39]:
test_tokenized

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})

In [40]:
len(test_tokenized[0]['input_ids'])

90

In [41]:
import torch.nn.functional as F

class OptimizedDataCollator:

    def __init__(self, tokenizer, pad_token_id=None, label_pad_token_id=-100, pad_to_multiple_of=8):
        self.tokenizer = tokenizer
        self.pad_token_id = pad_token_id if pad_token_id is not None else tokenizer.pad_token_id
        self.label_pad_token_id = label_pad_token_id
        self.pad_to_multiple_of = pad_to_multiple_of
    
    def __call__(self, features):
        """
        Efficiently pad and convert lists directly to tensors.
        """
        # Extract sequences (should all be lists now)
        input_ids = [feature["input_ids"] for feature in features]
        attention_masks = [feature["attention_mask"] for feature in features]
        labels = [feature["labels"] for feature in features]
        
        # Find max length for padding
        max_length = max(len(seq) for seq in input_ids)
        
        # Round up to multiple of pad_to_multiple_of for tensor core optimization
        if self.pad_to_multiple_of:
            max_length = ((max_length + self.pad_to_multiple_of - 1) 
                         // self.pad_to_multiple_of * self.pad_to_multiple_of)
        
        # Pad sequences efficiently
        padded_input_ids = []
        padded_attention_masks = []
        padded_labels = []
        
        for i in range(len(features)):
            # Calculate padding needed
            seq_len = len(input_ids[i])
            padding_length = max_length - seq_len
            
            # Pad input_ids
            padded_input_ids.append(
                input_ids[i] + [self.pad_token_id] * padding_length
            )
            
            # Pad attention_mask
            padded_attention_masks.append(
                attention_masks[i] + [0] * padding_length
            )
            
            # Pad labels
            padded_labels.append(
                labels[i] + [self.label_pad_token_id] * padding_length
            )
        
        # Convert to tensors efficiently (all at once)
        batch = {
            "input_ids": torch.tensor(padded_input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(padded_attention_masks, dtype=torch.long),
            "labels": torch.tensor(padded_labels, dtype=torch.long),
        }
        
        return batch



# Setup tokenizer padding
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id

# Create data collator
data_collator = OptimizedDataCollator(
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id,
    label_pad_token_id=-100,
    pad_to_multiple_of=config.PAD_TO_MULTIPLE_OF,
)

In [42]:
train_tokenized

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [43]:
def create_dataloaders(train_ds, val_ds, test_ds, data_collator, config):
    """Creates optimized data loaders for P100 GPU"""
    
    train_loader = DataLoader(
        train_ds,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        collate_fn=data_collator,
        num_workers=0,  
        pin_memory=True,
        drop_last=True, 
    )
    
    eval_loader = DataLoader(
        val_ds,
        batch_size=config.BATCH_SIZE ,  
        shuffle=False,
        collate_fn=data_collator,
        num_workers=0,
        pin_memory=True,
    )
    
    test_loader = DataLoader(
        test_ds,
        batch_size=config.BATCH_SIZE ,
        shuffle=False,
        collate_fn=data_collator,
        num_workers=0,
        pin_memory=True,
    )
    
    return train_loader, eval_loader, test_loader


train_loader, eval_loader, test_loader = create_dataloaders(
    train_tokenized, validation_tokenized, test_tokenized, data_collator, config
)

print(f"Data loaders created:")
print(f"Train batches: {len(train_loader)}")
print(f"Eval batches: {len(eval_loader)}")
print(f"Test batches: {len(test_loader)}")

Data loaders created:
Train batches: 5000
Eval batches: 500
Test batches: 100


In [44]:
#model has LoRA adapters added, but the original weights are frozen (not trainable). Only the adapters will learn during training, saving memory and time.

In [45]:
# creating the final model = base model + Adapter
def setup_lora_model(model, config):
    """
    Configures and applies LoRA to the model.
    
    LoRA works by decomposing weight updates into two smaller matrices:
    Instead of updating W directly, we learn ΔW = A × B where A and B are much smaller.
    """   
    print(" Setting up LoRA configuration...")
    
    peft_config = LoraConfig(
        r=config.LORA_RANK, 
        lora_alpha=config.LORA_ALPHA,  
        lora_dropout=config.LORA_DROPOUT, 
        bias="none", 
        target_modules="all-linear", 
        task_type="CAUSAL_LM", 
    )
    
    # Apply LoRA to model
    model = get_peft_model(model, peft_config)
    
    # Print trainable parameters info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f" LoRA setup complete!")
    print(f"   Total parameters: {total_params:,}")
    print(f"   Trainable parameters: {trainable_params:,}")
    print(f"   Trainable %: {100 * trainable_params / total_params:.2f}%")
    
    return model

# Apply LoRA
model = setup_lora_model(model, config)

 Setting up LoRA configuration...
 LoRA setup complete!
   Total parameters: 601,096,192
   Trainable parameters: 5,046,272
   Trainable %: 0.84%


In [46]:

def setup_optimizer_and_scheduler(model, train_loader, config):
    """
    Creates optimizer with proper weight decay and learning rate scheduler.
    """    
    # Get trainable parameters only (LoRA adapters)
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    
    # Separate parameters that should/shouldn't have weight decay
    decay_params = []
    no_decay_params = []
    
    for name, param in model.named_parameters():
        if param.requires_grad:
            if any(nd in name for nd in ["bias", "layer_norm", "layernorm"]):
                no_decay_params.append(param)
            else:
                decay_params.append(param)
    
    # Create parameter groups
    optimizer_grouped_parameters = [
        {
            "params": decay_params,
            "weight_decay": config.WEIGHT_DECAY,
        },
        {
            "params": no_decay_params,
            "weight_decay": 0.0,
        }
    ]
    
    # Create optimizer
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=config.LEARNING_RATE,
        betas=(config.BETA1, config.BETA2),
        eps=config.EPS,
    )
    
    # Calculate total training steps
    total_steps = len(train_loader) * config.NUM_EPOCHS // config.GRADIENT_ACCUMULATION_STEPS
    
    # Create learning rate scheduler
    scheduler = CosineAnnealingLR(
        optimizer,
        T_max=total_steps,
        eta_min=config.LEARNING_RATE * 0.1  # End at 10% of initial LR
    )
    
    print(f" Optimizer and scheduler setup:")
    print(f"   Learning rate: {config.LEARNING_RATE}")
    print(f"   Total training steps: {total_steps}")
    print(f"   Warmup steps: {config.WARMUP_STEPS}")
    
    return optimizer, scheduler, total_steps

optimizer, scheduler, total_steps = setup_optimizer_and_scheduler(model, train_loader, config)

 Optimizer and scheduler setup:
   Learning rate: 0.0002
   Total training steps: 1666
   Warmup steps: 100


In [47]:
def init_wandb(config, model):
    """Initialize Weights & Biases for experiment tracking"""
    
    wandb.init(
        project="lora-marathi-translation",
        name=f"lora-r{config.LORA_RANK}-alpha{config.LORA_ALPHA}-{datetime.now().strftime('%Y%m%d_%H%M')}",
        config={
            # LoRA settings
            "lora_rank": config.LORA_RANK,
            "lora_alpha": config.LORA_ALPHA, 
            "lora_dropout": config.LORA_DROPOUT,
            
            # Training settings
            "learning_rate": config.LEARNING_RATE,
            "batch_size": config.BATCH_SIZE,
            "gradient_accumulation_steps": config.GRADIENT_ACCUMULATION_STEPS,
            "num_epochs": config.NUM_EPOCHS,
            "max_grad_norm": config.MAX_GRAD_NORM,
            
            # Model info
            "model_name": model.config.name_or_path if hasattr(model.config, 'name_or_path') else "unknown",
            "total_params": sum(p.numel() for p in model.parameters()),
            "trainable_params": sum(p.numel() for p in model.parameters() if p.requires_grad),
        }
    )
    

init_wandb(config, model)

In [48]:
def evaluate_model(model, dataloader, device, desc="Evaluating"):
    """
    Comprehensive model evaluation with multiple metrics.
    """
    model.eval()
    total_loss = 0.0
    total_tokens = 0
    losses = []
    
    eval_bar = tqdm(dataloader, desc=desc, leave=False)
    
    with torch.no_grad():
        for batch in eval_bar:
            # Move to device
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
            
            # Forward pass
            outputs = model(**batch)
            loss = outputs.loss
            
            # Calculate metrics
            batch_loss = loss.item()
            batch_tokens = (batch["labels"] != -100).sum().item()
            
            total_loss += batch_loss * batch_tokens
            total_tokens += batch_tokens
            losses.append(batch_loss)
            
            eval_bar.set_postfix({
                "loss": f"{batch_loss:.4f}",
                "avg_loss": f"{total_loss/total_tokens:.4f}"
            })
    
    avg_loss = total_loss / total_tokens if total_tokens > 0 else float('inf')
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    
    return {
        "avg_loss": avg_loss,
        "perplexity": perplexity,
        "total_tokens": total_tokens,
        "num_batches": len(losses)
    }




In [49]:
def save_checkpoint(model, tokenizer, optimizer, scheduler, step, loss, output_dir, is_best=False, epoch=None):  # Add epoch param
       """Save model checkpoint with improved management - only keep latest + best"""
       scaler = GradScaler()
       if is_best:
           checkpoint_name = "best-checkpoint"
       else:
           checkpoint_name = "latest-checkpoint"  
       
       checkpoint_dir = os.path.join(output_dir, checkpoint_name)
       
       # Remove existing latest checkpoint before saving new one
       if not is_best and os.path.exists(checkpoint_dir):
           try:
               shutil.rmtree(checkpoint_dir)
               print(f"🗑️ Removed previous latest checkpoint")
           except OSError as e:
               print(f"Warning: Could not remove old checkpoint: {e}")
       
       os.makedirs(checkpoint_dir, exist_ok=True)
       
       # Save model and tokenizer
       model.save_pretrained(checkpoint_dir)
       tokenizer.save_pretrained(checkpoint_dir)
       
       # Save training state
       torch.save({
           'step': step,
           'epoch': epoch,  # NEW: Save current epoch (0-based)
           'model_state_dict': model.state_dict(),
           'optimizer_state_dict': optimizer.state_dict(),
           'scheduler_state_dict': scheduler.state_dict(),
           'loss': loss,
           'scaler_state_dict': scaler.state_dict(),
       }, os.path.join(checkpoint_dir, 'training_state.pt'))
       
       # Save metadata
       metadata = {
           "step": step,
           "epoch": epoch,  # NEW
           "loss": loss,
           "timestamp": datetime.now().isoformat(),
           "is_best": is_best
       }
       
       with open(os.path.join(checkpoint_dir, 'metadata.json'), 'w') as f:
           json.dump(metadata, f, indent=2)
       
       if is_best:
           print(f" Best checkpoint saved: {checkpoint_dir}")
       else:
           print(f" Latest checkpoint saved: {checkpoint_dir}")
       
       return checkpoint_dir

In [50]:

def train_model():
    """
    Main training loop with comprehensive monitoring and checkpointing.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f" Training on device: {device}")
    
    # Move model to device
    model.to(device)
    
    # Training tracking variables
    global_step = 0
    best_eval_loss = float('inf')
    saved_checkpoints = []
    
    # Create output directory
    output_dir = f"lora-checkpoints-{datetime.now().strftime('%Y%m%d_%H%M')}"
    os.makedirs(output_dir, exist_ok=True)
    
    print(f" Starting training for {config.NUM_EPOCHS} epochs")
    print(f"   Effective batch size: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
    print(f"   Total steps: {total_steps}")
    print(f"   Checkpoints saved to: {output_dir}")
    scaler = GradScaler()
    for epoch in range(config.NUM_EPOCHS):
        model.train()
        
        # Epoch tracking
        epoch_loss = 0.0
        epoch_steps = 0
        accumulated_loss = 0.0
        
        # Progress bar for epoch
        epoch_bar = tqdm(
            train_loader, 
            desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS}",
            total=len(train_loader)
        )
        
        for step, batch in enumerate(epoch_bar):
            # Move batch to device
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}

            with torch.cuda.amp.autocast():
                
                # NEW – fp16 forward/backward
                outputs = model(**batch)
                loss = outputs.loss
                loss = loss / config.GRADIENT_ACCUMULATION_STEPS

            
            # # Forward pass
            # outputs = model(**batch)
            # loss = outputs.loss
            
            # # Scale loss for gradient accumulation
            # loss = loss / config.GRADIENT_ACCUMULATION_STEPS
            scaler.scale(loss).backward()
            # Backward pass
            
            
            # Accumulate loss
            accumulated_loss += loss.item()
            epoch_loss += loss.item()
            
            # Update weights every GRADIENT_ACCUMULATION_STEPS
            if (step + 1) % config.GRADIENT_ACCUMULATION_STEPS == 0:
                # Clip gradients
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.MAX_GRAD_NORM)

                torch.nn.utils.clip_grad_norm_(model.parameters(), config.MAX_GRAD_NORM)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
                

                global_step += 1
                
                # Log to wandb
                current_lr = scheduler.get_last_lr()[0]
                wandb.log({
                    "train/loss": accumulated_loss,
                    "train/learning_rate": current_lr,
                    "train/epoch": epoch + (step + 1) / len(train_loader),
                    "train/global_step": global_step
                }, step=global_step)
                
                # Update progress bar
                epoch_bar.set_postfix({
                    "loss": f"{accumulated_loss:.4f}",
                    "lr": f"{current_lr:.2e}",
                    "step": global_step
                })
                
                # Reset accumulated loss
                accumulated_loss = 0.0
                
                # Evaluation
                if global_step % config.EVAL_EVERY_N_STEPS == 0:
                    print(f"\n Running evaluation at step {global_step}")
                    
                    eval_results = evaluate_model(model, eval_loader, device, "Validation")
                    
                    # Log evaluation results
                    wandb.log({
                        "eval/loss": eval_results["avg_loss"],
                        "eval/perplexity": eval_results["perplexity"],
                        "eval/tokens": eval_results["total_tokens"]
                    }, step=global_step)
                    
                    print(f"   Validation loss: {eval_results['avg_loss']:.4f}")
                    print(f"   Perplexity: {eval_results['perplexity']:.2f}")
                    
                    # Save best model
                    if eval_results["avg_loss"] < best_eval_loss:
                        best_eval_loss = eval_results["avg_loss"]
                        save_checkpoint(
                            model, tokenizer, optimizer, scheduler,
                            global_step, eval_results["avg_loss"], output_dir, is_best=True
                        )
                        wandb.log({"eval/best_loss": best_eval_loss}, step=global_step)
                    
                    model.train()  # Back to training mode
                
                # Save regular checkpoint
                if global_step % config.SAVE_EVERY_N_STEPS == 0:
                    checkpoint_dir = save_checkpoint(
                        model, tokenizer, optimizer, scheduler,
                        global_step, accumulated_loss, output_dir, is_best=False
                    )
            
            epoch_steps += 1
            
            # Memory cleanup
            if step % 50 == 0:
                torch.cuda.empty_cache()
                gc.collect()
        
        # End of epoch summary
        avg_epoch_loss = epoch_loss / epoch_steps
        print(f"\n Epoch {epoch+1} Summary:")
        print(f"   Average loss: {avg_epoch_loss:.4f}")
        print(f"   Steps completed: {epoch_steps}")
        print(f"   Global step: {global_step}")
        
        wandb.log({
            "epoch/avg_loss": avg_epoch_loss,
            "epoch/number": epoch + 1
        }, step=global_step)
    
    # Final evaluation
    print("\n Final evaluation...")
    final_eval = evaluate_model(model, eval_loader, device, "Final Evaluation")
    test_eval = evaluate_model(model, test_loader, device, "Test Evaluation")
    
    # Log final results
    wandb.log({
        "final/eval_loss": final_eval["avg_loss"],
        "final/eval_perplexity": final_eval["perplexity"],
        "final/test_loss": test_eval["avg_loss"],
        "final/test_perplexity": test_eval["perplexity"]
    })
    
    print(f" Final Results:")
    print(f"   Best validation loss: {best_eval_loss:.4f}")
    print(f"   Final validation loss: {final_eval['avg_loss']:.4f}")
    print(f"   Test loss: {test_eval['avg_loss']:.4f}")
    print(f"   Test perplexity: {test_eval['perplexity']:.2f}")
    
    # Save final model
    final_checkpoint = save_checkpoint(
        model, tokenizer, optimizer, scheduler,
        global_step, final_eval["avg_loss"], output_dir
    )
    
    print(f" Training completed! Models saved in: {output_dir}")
    return output_dir, best_eval_loss

# Run training
output_dir, best_loss = train_model()

 Training on device: cuda
 Starting training for 1 epochs
   Effective batch size: 6
   Total steps: 1666
   Checkpoints saved to: lora-checkpoints-20250713_1825


/tmp/ipykernel_36/2233922602.py:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/1:   0%|          | 0/5000 [00:00<?, ?it/s]

/tmp/ipykernel_36/2233922602.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():



 Running evaluation at step 1000


...

/tmp/ipykernel_36/3401146592.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


 Best checkpoint saved: lora-checkpoints-20250713_1825/best-checkpoint

 Epoch 1 Summary:
   Average loss: 0.4291
   Steps completed: 5000
   Global step: 1666

 Final evaluation...


Final Evaluation:   0%|          | 0/500 [00:00<?, ?it/s]

Test Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

 Final Results:
   Best validation loss: 1.2388
   Final validation loss: 1.2088
   Test loss: 1.2244
   Test perplexity: 3.40
 Latest checkpoint saved: lora-checkpoints-20250713_1825/latest-checkpoint
 Training completed! Models saved in: lora-checkpoints-20250713_1825


In [51]:
def test_model_inference(model, tokenizer, device, test_cases=None):
    """Test the fine-tuned model with sample inputs"""
    
    if test_cases is None:
        test_cases = [
            {
                "instruction": "Translate the following English text to Marathi:",
                "english": "Good morning, how are you today?",
            },
            {
                "instruction": "Convert this English sentence to Marathi:",
                "english": "I love learning new languages.",
            }
        ]
    
    model.eval()
    
    print(" Testing model inference...")
    
    for i, test_case in enumerate(test_cases):
        prompt = f"{test_case['instruction']} English: {test_case['english']} Marathi:"
        
        # Tokenize input
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=config.MAX_LENGTH // 2
        ).to(device)
        
        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        # Decode response
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract just the Marathi translation
        marathi_translation = generated_text[len(prompt):].strip()
        
        print(f"\n Test Case {i+1}:")
        print(f"English: {test_case['english']}")
        print(f"Generated Marathi: {marathi_translation}")

# Test the model
test_model_inference(model, tokenizer, device)

 Testing model inference...

 Test Case 1:
English: Good morning, how are you today?
Generated Marathi: माझ्या दिवसाने काय करत आहोत?

 Test Case 2:
English: I love learning new languages.
Generated Marathi: या भाषेचा ज्यासाठी मनाचा सुद्धा बनवावा.


In [52]:
import torch
import evaluate
from tqdm.auto import tqdm

def evaluate_model_bleu(model, tokenizer, df_test, batch_size=16):
    print("Evaluating model and calculating SacreBLEU score...")
    metric = evaluate.load("sacrebleu")
    model.eval()  # Set model to inference mode

    # This is a standard practice for decoder-only models.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    predictions, refs = [], []

    for i in tqdm(range(0, len(df_test), batch_size), desc="Generating Predictions"):
        batch = df_test[i : i + batch_size]
        src_sentences = batch["english"]
        ref_sentences = batch["marathi"]

        prompts = [
            f"Convert the English text into Marathi language. English: {src} Marathi:"
            for src in src_sentences
        ]

        # Tokenize inputs
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
            padding_side='left', 
        ).to(model.device)

        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,  # Use greedy decoding
                eos_token_id=tokenizer.eos_token_id,
            )

        # Efficiently slice and decode the entire batch at once
        prompt_token_length = inputs.input_ids.shape[1]
        generated_only_ids = generated[:, prompt_token_length:]  # Corrected line

        # Use batch_decode for faster processing
        decoded_preds = tokenizer.batch_decode(generated_only_ids, skip_special_tokens=True)

        # Strip any extra whitespace from each prediction
        predictions.extend([pred.strip() for pred in decoded_preds])
        refs.extend([[t] for t in ref_sentences])

    # Compute and return the final BLEU score
    bleu_result = metric.compute(predictions=predictions, references=refs)
    return bleu_result

# Compile model for faster inference
print("Compiling model for faster inference...")
model = torch.compile(model)

# --- How to use the function ---
# Assuming your model, tokenizer, and df_test are loaded
bleu_score = evaluate_model_bleu(model, tokenizer, df_test)
print(f"SacreBLEU on test set: {bleu_score['score']:.2f}")

Compiling model for faster inference...
Evaluating model and calculating SacreBLEU score...


Generating Predictions:   0%|          | 0/13 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

SacreBLEU on test set: 0.29


In [53]:
def training_summary():
    
    print("\n" + "="*50)
    print("🎉 TRAINING COMPLETE!")
    print("="*50)
    
    print(f" Training Configuration:")
    print(f"   LoRA Rank: {config.LORA_RANK}")
    print(f"   LoRA Alpha: {config.LORA_ALPHA}")
    print(f"   Learning Rate: {config.LEARNING_RATE}")
    print(f"   Batch Size: {config.BATCH_SIZE}")
    print(f"   Epochs: {config.NUM_EPOCHS}")
    
    print(f"\n Model saved to: {output_dir}")
    print(f"Best validation loss: {best_loss:.4f}")
    
    print(f"\nView training progress at: {wandb.run.url}")
    
    print("\nNext steps:")
    print("   1. Test model on new examples")
    print("   2. Fine-tune hyperparameters if needed")
    print("   3. Deploy model for inference")
    print("   4. Create LoRA adapters for other tasks")

training_summary()

# Finish wandb run
wandb.finish()

# Memory cleanup
torch.cuda.empty_cache()
gc.collect()
print("\nMemory cleanup completed!")


🎉 TRAINING COMPLETE!
 Training Configuration:
   LoRA Rank: 8
   LoRA Alpha: 16
   Learning Rate: 0.0002
   Batch Size: 2
   Epochs: 1

 Model saved to: lora-checkpoints-20250713_1825
Best validation loss: 1.2388

View training progress at: https://wandb.ai/22cs3033-rgipt-jais/lora-marathi-translation/runs/2kfoqqwb

Next steps:
   1. Test model on new examples
   2. Fine-tune hyperparameters if needed
   3. Deploy model for inference
   4. Create LoRA adapters for other tasks


epoch/avg_loss,▁
epoch/number,▁
eval/best_loss,▁
eval/loss,▁
eval/perplexity,▁
eval/tokens,▁
final/eval_loss,▁
final/eval_perplexity,▁
final/test_loss,▁
final/test_perplexity,▁
train/epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇████



Memory cleanup completed!


In [54]:
# Memory cleanup
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
def resume_training(
    resume_from_checkpoint: str,
    base_model_name: str = "Qwen/Qwen3-0.6B",
    resume_from_epoch: int = 2,
    additional_epochs: int = 2,
     output_dir: str | None = None,  
):


    import os, json, gc, evaluate, wandb, torch
    from tqdm import tqdm
    from datetime import datetime
    from peft import PeftModel
    from torch.cuda.amp import GradScaler, autocast
    from transformers import AutoModelForCausalLM, AutoTokenizer

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(
        f" Resuming training from: {resume_from_checkpoint}\n"
        f"   Starting at epoch {resume_from_epoch} for {additional_epochs} more."
    )

    output_dir = (
        os.path.dirname(resume_from_checkpoint)
        if "latest-checkpoint" in resume_from_checkpoint
        else resume_from_checkpoint
    )
    training_state_path = os.path.join(resume_from_checkpoint, "training_state.pt")
    if not os.path.exists(training_state_path):
        raise FileNotFoundError(f"No training_state.pt found in {resume_from_checkpoint}")

    training_state = torch.load(training_state_path, map_location="cpu")
    global_step_saved = training_state["step"]

    required_vocab_size = training_state["model_state_dict"][
        "base_model.model.model.embed_tokens.weight"
    ].shape[0]
    print(f"Required vocab size in checkpoint: {required_vocab_size}")

    tokenizer = AutoTokenizer.from_pretrained(resume_from_checkpoint)
    if len(tokenizer) < required_vocab_size:
        extra = required_vocab_size - len(tokenizer)
        tokenizer.add_tokens([f"<extra_id_{i}>" for i in range(extra)])
        print(f"➕ Added {extra} dummy tokens (tokenizer now {len(tokenizer)})")


    base_model = AutoModelForCausalLM.from_pretrained(base_model_name, low_cpu_mem_usage=True)
    base_model.resize_token_embeddings(len(tokenizer))
    base_model.config.pad_token_id = tokenizer.pad_token_id

    model = PeftModel.from_pretrained(base_model, resume_from_checkpoint, is_trainable=True)
    model.load_state_dict(training_state["model_state_dict"])
    model.to(device)
    print("Model & tokenizer loaded – all shapes match")


    config.NUM_EPOCHS = (resume_from_epoch - 1) + additional_epochs
    optimizer, scheduler, total_steps = setup_optimizer_and_scheduler(model, train_loader, config)

    saved_opt_state = training_state["optimizer_state_dict"]
    if len(optimizer.param_groups) != len(saved_opt_state["param_groups"]):
        raise ValueError("Parameter-group mismatch with saved optimizer")

    # remap IDs → tensors
    new_state = {}
    cur_params = [p for g in optimizer.param_groups for p in g["params"]]
    saved_param_ids = [pid for g in saved_opt_state["param_groups"] for pid in g["params"]]
    if len(cur_params) != len(saved_param_ids):
        raise ValueError("Trainable-parameter count changed")

    for p_cur, p_id_saved in zip(cur_params, saved_param_ids):
        if p_id_saved in saved_opt_state["state"]:
            new_state[p_cur] = saved_opt_state["state"][p_id_saved]
    optimizer.state = new_state

    def _optimizer_to(opt, dev):
        """Move all tensors in optimizer.state to device `dev`"""
        for st in opt.state.values():
            for k, v in st.items():
                if isinstance(v, torch.Tensor):
                    st[k] = v.to(dev)
        return opt

    optimizer = _optimizer_to(optimizer, device)
    print("Optimizer state restored & moved to", device)

    scheduler.load_state_dict(training_state["scheduler_state_dict"])
    scheduler.T_max = total_steps

    scaler = GradScaler()
    scaler.load_state_dict(training_state["scaler_state_dict"])


    best_eval_loss = float("inf")
    meta_path = os.path.join(output_dir, "best-checkpoint", "metadata.json")
    if os.path.exists(meta_path):
        best_eval_loss = json.load(open(meta_path))["loss"]
    print(f"Best eval loss so far: {best_eval_loss:.4f} | global step: {global_step_saved}")

    # WandB 
    init_wandb(config, model)

    # training loop 
    global_step = global_step_saved
    start_epoch = resume_from_epoch - 1

    for epoch in range(start_epoch, config.NUM_EPOCHS):
        model.train()
        epoch_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS}")

        acc_loss = tot_loss = steps = 0.0
        for step, batch in enumerate(epoch_bar):
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}

            with autocast():
                loss = model(**batch).loss / config.GRADIENT_ACCUMULATION_STEPS
            scaler.scale(loss).backward()

            acc_loss += loss.item()
            tot_loss += loss.item()

            if (step + 1) % config.GRADIENT_ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.MAX_GRAD_NORM)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()

                global_step += 1
                lr_now = scheduler.get_last_lr()[0]
                wandb.log(
                    {
                        "train/loss": acc_loss,
                        "train/lr": lr_now,
                        "train/epoch": epoch + (step + 1) / len(train_loader),
                        "train/step": global_step,
                    },
                    step=global_step,
                )
                epoch_bar.set_postfix(loss=f"{acc_loss:.4f}", lr=f"{lr_now:.2e}")
                acc_loss = 0.0

                if global_step % config.EVAL_EVERY_N_STEPS == 0:
                    eval_stats = evaluate_model(model, eval_loader, device, "Validation")
                    wandb.log(
                        {
                            "eval/loss": eval_stats["avg_loss"],
                            "eval/perplexity": eval_stats["perplexity"],
                        },
                        step=global_step,
                    )
                    if eval_stats["avg_loss"] < best_eval_loss:
                        best_eval_loss = eval_stats["avg_loss"]
                        save_checkpoint(
                            model, tokenizer, optimizer, scheduler,
                            global_step, best_eval_loss, output_dir, is_best=True,
                        )

                if global_step % config.SAVE_EVERY_N_STEPS == 0:
                    save_checkpoint(
                        model, tokenizer, optimizer, scheduler,
                        global_step, loss.item(), output_dir, is_best=False,
                    )

            steps += 1
            if step % 50 == 0:
                torch.cuda.empty_cache()
                gc.collect()

        print(f"Epoch {epoch+1} done | avg loss: {tot_loss/steps:.4f} | global step: {global_step}")
        wandb.log({"epoch/avg_loss": tot_loss / steps}, step=global_step)

    # final evaluation 
    final_eval = evaluate_model(model, eval_loader, device, "Final Evaluation")
    test_eval = evaluate_model(model, test_loader, device, "Test Evaluation")
    wandb.log(
        {
            "final/val_loss": final_eval["avg_loss"],
            "final/test_loss": test_eval["avg_loss"],
            "final/test_perplexity": test_eval["perplexity"],
        },
        step=global_step,
    )

    # decide where to save
    if output_dir is None:
        output_dir = (
            os.path.dirname(resume_from_checkpoint)
            if "latest-checkpoint" in resume_from_checkpoint
            else resume_from_checkpoint
        )
    if not os.access(output_dir, os.W_OK):
        ts = datetime.now().strftime("%Y%m%d_%H%M")
        output_dir = f"/kaggle/working/lora-resume-{ts}"
        print(f"Switched output_dir to writable location: {output_dir}")
    os.makedirs(output_dir, exist_ok=True)

    save_checkpoint(model, tokenizer, optimizer, scheduler, global_step, final_eval["avg_loss"], output_dir)

    print("🔧 Compiling model for inference …")
    model = torch.compile(model)
    bleu = evaluate_model_bleu(model, tokenizer, df_test)
    print(f"SacreBLEU: {bleu['score']:.2f}")

    wandb.finish()
    torch.cuda.empty_cache()
    gc.collect()
    print("Resumed training complete. Models in:", output_dir)

In [ ]:
resume_from_checkpoint = "/kaggle/input/qwen3-finetuned/transformers/default/3/lora-checkpoints-20250712_1620/latest-checkpoint"
resume_training(resume_from_checkpoint, base_model_name="Qwen/Qwen3-0.6B", resume_from_epoch=2, additional_epochs=2 ,  output_dir="/kaggle/working/lora-checkpoints-resume")